# Processing

In [18]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# Ensure plots are displayed within the notebook
%matplotlib inline

In [19]:
# Load the dataset
df = pd.read_csv("data/data.csv")

In [20]:
df.head()

,FL_DATE,AIRLINE,AIRLINE_DOT,AIRLINE_CODE,DOT_CODE,FL_NUMBER,ORIGIN,ORIGIN_CITY,DEST,DEST_CITY,...,DIVERTED,CRS_ELAPSED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,DELAY_DUE_CARRIER,DELAY_DUE_WEATHER,DELAY_DUE_NAS,DELAY_DUE_SECURITY,DELAY_DUE_LATE_AIRCRAFT
0,2019-01-09,United Air Lines Inc.,United Air Lines Inc.: UA,UA,19977,1562,FLL,"Fort Lauderdale, FL",EWR,"Newark, NJ",...,0.0,186.0,176.0,153.0,1065.0,NaN,NaN,NaN,NaN,NaN
1,2022-11-19,Delta Air Lines Inc.,Delta Air Lines Inc.: DL,DL,19790,1149,MSP,"Minneapolis, MN",SEA,"Seattle, WA",...,0.0,235.0,236.0,189.0,1399.0,NaN,NaN,NaN,NaN,NaN
2,2022-07-22,United Air Lines Inc.,United Air Lines Inc.: UA,UA,19977,459,DEN,"Denver, CO",MSP,"Minneapolis, MN",...,0.0,118.0,112.0,87.0,680.0,NaN,NaN,NaN,NaN,NaN
3,2023-03-06,Delta Air Lines Inc.,Delta Air Lines Inc.: DL,DL,19790,2295,MSP,"Minneapolis, MN",SFO,"San Francisco, CA",...,0.0,260.0,285.0,249.0,1589.0,0.0,0.0,24.0,0.0,0.0
4,2020-02-23,Spirit Air Lines,Spirit Air Lines: NK,NK,20416,407,MCO,"Orlando, FL",DFW,"Dallas/Fort Worth, TX",...,0.0,181.0,182.0,153.0,985.0,NaN,NaN,NaN,NaN,NaN


In [21]:
# Define the 30 largest airports by passenger traffic, ensuring geographical distribution
largest_airports = [
    'ATL',  # Atlanta, GA (Southeast)
    'LAX',  # Los Angeles, CA (West)
    'ORD',  # Chicago, IL (Midwest)
    'DFW',  # Dallas/Fort Worth, TX (Southwest)
    'DEN',  # Denver, CO (Mountain)
    'JFK',  # New York, NY (Northeast)
    'SEA',  # Seattle, WA (Northwest)
    'MIA',  # Miami, FL (Southeast)
    'PHX',  # Phoenix, AZ (Southwest)
    'BOS',  # Boston, MA (Northeast)
    'MSP',  # Minneapolis, MN (Midwest)
    'CLT',  # Charlotte, NC (Southeast)
    'LAS',  # Las Vegas, NV (West)
    'IAH',  # Houston, TX (South)
    'SFO',  # San Francisco, CA (West)
    'EWR',  # Newark, NJ (Northeast)
    'FLL',  # Fort Lauderdale, FL (Southeast)
    'DTW',  # Detroit, MI (Midwest)
    'PHL',  # Philadelphia, PA (Northeast)
    'BWI',  # Baltimore, MD (Northeast)
    'SLC',  # Salt Lake City, UT (Mountain)
    'SAN',  # San Diego, CA (West)
    'TPA',  # Tampa, FL (Southeast)
    'PDX',  # Portland, OR (Northwest)
    'MCO',  # Orlando, FL (Southeast)
    'HNL',  # Honolulu, HI (Pacific)
    'DCA',  # Washington, D.C. (Northeast)
    'STL',  # St. Louis, MO (Midwest)
    'AUS',  # Austin, TX (South)
    'BNA'   # Nashville, TN (South)
]

In [22]:
import pandas as pd

# Assuming `df` is your DataFrame and `largest_airports` is defined
# Step 1: Filter for flights that originate AND land at the 30 largest airports
df = df[(df['ORIGIN'].isin(largest_airports)) & (df['DEST'].isin(largest_airports))]
print(f"Rows after filtering by airports: {len(df)}")

if 'DEP_HOUR' not in df.columns:
    df['DEP_HOUR'] = df['CRS_DEP_TIME'] // 100

# Step 2: Remove canceled flights
df = df[df['CANCELLED'] == 0]
print(f"Rows after removing canceled flights: {len(df)}")

# Step 3: Remove flights with extreme delays (longer than 8 hours or 480 minutes)
df = df[df['ARR_DELAY'] <= 180]
print(f"Rows after removing extreme delays: {len(df)}")

# Step 4: Convert FL_DATE to datetime and filter out flights from 2020, 2022, and 2023
df['FL_DATE'] = pd.to_datetime(df['FL_DATE'], errors='coerce')
df = df[~df['FL_DATE'].dt.year.isin([2020, 2022, 2023])]  # Exclude 2020, 2022, and 2023
print(f"Rows after filtering out flights from 2020, 2022, and 2023: {len(df)}")

# Step 5: Convert the FL_DATE column into just the month as a numerical value
df.loc[:, 'MONTH'] = df['FL_DATE'].dt.month

# Step 6: Drop rows where 'DELAY_DUE_SECURITY' is greater than 0, but keep rows where it is NaN or 0 (no delay)
df = df[(df['DELAY_DUE_SECURITY'].isna()) | (df['DELAY_DUE_SECURITY'] == 0)]
print(f"Rows after removing delayed to security: {len(df)}")

# Feature Engineering
# 1. Extract hour of the day from CRS_DEP_TIME (assuming CRS_DEP_TIME is in HHMM format)
df.loc[:, 'DEP_HOUR'] = df['CRS_DEP_TIME'] // 100

# 2. Extract the day of the week from FL_DATE
df.loc[:, 'DAY_OF_WEEK'] = df['FL_DATE'].dt.dayofweek

# Step 7: Drop unnecessary columns
df = df.drop(columns=['FL_DATE', 'DEP_DELAY', 'TAXI_OUT', 'TAXI_IN', 'WHEELS_ON', 'WHEELS_OFF', 'CANCELLED',
                      'CANCELLATION_CODE', 'DIVERTED', 'CRS_DEP_TIME', 'DEP_TIME', 'CRS_ARR_TIME', 'ARR_TIME',
                      'CRS_ELAPSED_TIME', 'AIRLINE_DOT', 'AIRLINE_CODE', 'DOT_CODE', 'ORIGIN_CITY', 'DEST_CITY',
                      'DELAY_DUE_CARRIER', 'DELAY_DUE_WEATHER', 'DELAY_DUE_NAS', 'DELAY_DUE_SECURITY',
                      'DELAY_DUE_LATE_AIRCRAFT'])

# Step 8: Handle NaN values by dropping rows with missing data
df = df.dropna()
print(f"Rows after dropping NaN values: {len(df)}")

# Step 9: Calculate monthly average delay and scale between -1 and 1
monthly_avg_delay = df.groupby('MONTH')['ARR_DELAY'].mean().reset_index()
monthly_avg_delay.columns = ['MONTH', 'MONTHLY_AVG_DELAY']
# Scale the monthly average delay between -1 and 1
min_monthly_delay = monthly_avg_delay['MONTHLY_AVG_DELAY'].min()
max_monthly_delay = monthly_avg_delay['MONTHLY_AVG_DELAY'].max()
monthly_avg_delay['MONTHLY_DELAY_INDICATOR'] = 2 * (monthly_avg_delay['MONTHLY_AVG_DELAY'] - min_monthly_delay) / (max_monthly_delay - min_monthly_delay) - 1
# Merge the scaled monthly indicator back into the main dataset
df = pd.merge(df, monthly_avg_delay[['MONTH', 'MONTHLY_DELAY_INDICATOR']], on='MONTH', how='left')

# Step 10: Calculate average arrival delay for each route (ORIGIN-DEST combination)
route_avg_delay = df.groupby(['ORIGIN', 'DEST'])['ARR_DELAY'].mean().reset_index()
route_avg_delay.columns = ['ORIGIN', 'DEST', 'ROUTE_AVG_DELAY']
# Scale the route average delay between -1 and 1
min_route_delay = route_avg_delay['ROUTE_AVG_DELAY'].min()
max_route_delay = route_avg_delay['ROUTE_AVG_DELAY'].max()
route_avg_delay['ROUTE_DELAY_INDICATOR'] = 2 * (route_avg_delay['ROUTE_AVG_DELAY'] - min_route_delay) / (max_route_delay - min_route_delay) - 1
# Merge the scaled route indicator back into the main dataset
df = pd.merge(df, route_avg_delay[['ORIGIN', 'DEST', 'ROUTE_DELAY_INDICATOR']], on=['ORIGIN', 'DEST'], how='left')

# Keep the final columns, including ARR_DELAY as the target variable and the monthly delay indicator
df = df[['AIRLINE', 'DAY_OF_WEEK', 'MONTH', 'DEST', 'ORIGIN', 'DEP_HOUR', 'DISTANCE', 'AIR_TIME', 'ELAPSED_TIME',
         'MONTHLY_DELAY_INDICATOR', 'ROUTE_DELAY_INDICATOR', 'ARR_DELAY']]
print(f"Final dataset rows: {len(df)}")

# Save the cleaned and engineered dataset to a CSV file locally
output_path = 'data_cleaned.csv'  # Local path
df.to_csv(output_path, index=False)
print(f"Feature engineering complete. Data saved to '{output_path}'")


Rows after filtering by airports: 1011735
Rows after removing canceled flights: 988255
Rows after removing extreme delays: 974387
Rows after filtering out flights from 2020, 2022, and 2023: 443213
Rows after removing delayed to security: 442760
Rows after dropping NaN values: 442760
Final dataset rows: 442760
Feature engineering complete. Data saved to 'data_cleaned.csv'


In [23]:
df.head()

,AIRLINE,DAY_OF_WEEK,MONTH,DEST,ORIGIN,DEP_HOUR,DISTANCE,AIR_TIME,ELAPSED_TIME,MONTHLY_DELAY_INDICATOR,ROUTE_DELAY_INDICATOR,ARR_DELAY
0,United Air Lines Inc.,2,1,EWR,FLL,11,1065.0,153.0,176.0,-1.000000,0.263236,-14.0
1,Spirit Air Lines,6,3,IAH,DEN,15,862.0,110.0,139.0,-0.877541,0.232799,-13.0
2,American Airlines Inc.,4,6,DFW,SLC,6,989.0,127.0,151.0,1.000000,0.148767,-11.0
3,American Airlines Inc.,5,11,ORD,DFW,18,801.0,106.0,131.0,-0.814371,0.187652,-9.0
4,Delta Air Lines Inc.,5,6,ATL,DTW,13,594.0,100.0,116.0,1.000000,-0.023691,-3.0
